Install Dependencies

In [1]:
# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

#Imports pickle
import pickle

#Library for indicators
from finta import TA

#Another library for indicators
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.trend import PSARIndicator
from ta.trend import SMAIndicator
from ta.trend import macd
from ta.momentum import rsi
from ta.volume import OnBalanceVolumeIndicator

Selects the dataframe for training

In [3]:
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/USDGBP.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/USDJPY.csv')
df = pd.read_csv('/Users/esteban/Documents/GitHub/Reinforcement_Learning_Binary_Options_Bot/BTCUSD_1hr.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/BTCUSD_1hr.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/BTCUSD_1min.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/Download Data - STOCK_US_XNYS_F (1).csv')

df

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,1586390400000,2020-04-09 00:00:00,BTCUSD,7369.60,7369.60,7338.23,7338.23,7.788915
1,1586386800000,2020-04-08 23:00:00,BTCUSD,7367.28,7398.92,7348.47,7369.60,66.558293
2,1586383200000,2020-04-08 22:00:00,BTCUSD,7337.19,7390.25,7322.87,7367.28,88.486108
3,1586379600000,2020-04-08 21:00:00,BTCUSD,7327.70,7370.00,7311.33,7337.19,35.551553
4,1586376000000,2020-04-08 20:00:00,BTCUSD,7320.11,7375.72,7313.66,7327.70,153.655811
...,...,...,...,...,...,...,...,...
39460,1444323600,2015-10-08 17:00:00,BTCUSD,244.25,244.99,244.02,244.99,3.920632
39461,1444320000,2015-10-08 16:00:00,BTCUSD,244.92,244.92,244.25,244.25,3.895252
39462,1444316400,2015-10-08 15:00:00,BTCUSD,245.00,245.00,244.92,244.92,3.016926
39463,1444312800,2015-10-08 14:00:00,BTCUSD,245.00,245.00,244.50,245.00,4.453649


Data Cleanup

In [5]:
print(df.dtypes)

#Convert date
#df['Date'] = pd.to_datetime(df['Date'])

#Sort values from past to present
df.sort_values('Date', ascending=True, inplace=True)

#Set date as index
df.set_index('Date', inplace=True)

#Converts volume from string to float
#df.drop('Volume', inplace=True, axis=1)
#df['Volume'] = df['Volume'].apply(lambda x: float(x.replace(",", "")))
#df['Volume'] = df['Volume'].astype(float)
#df = df.drop(columns=['Symbol'])
#df = df.drop(columns=['Unix Timestamp'])
df = df.drop(columns=['Volume USD'])
df = df.rename(columns={'Volume BTC': 'Volume'})

print(df.shape[0])
print(df.dtypes)

df.head()

Unix Timestamp      int64
Symbol             object
Open              float64
High              float64
Low               float64
Close             float64
Volume            float64
dtype: object


KeyError: "None of ['Date'] are in the columns"

Normalization

In [ ]:
'''
df["Open"] = df["Open"] - df["Open"].shift(1)
df["High"] = df["High"] - df["High"].shift(1)
df["Low"] = df["Low"] - df["Low"].shift(1)
df["Close"] = df["Close"] - df["Close"].shift(1)
'''

'\ndf["Open"] = df["Open"] - df["Open"].shift(1)\ndf["High"] = df["High"] - df["High"].shift(1)\ndf["Low"] = df["Low"] - df["Low"].shift(1)\ndf["Close"] = df["Close"] - df["Close"].shift(1)\n'

# 2.1 Add indicators
2.1.1 Install New Dependencies

2.1.3. Calculate SMA, RSI and OBV


In [6]:

#Indicators with TA
#Adds simple moving average
df["sma7"] = SMAIndicator(close=df["Open"], window=7, fillna=True).sma_indicator()
df["sma25"] = SMAIndicator(close=df["Open"], window=25, fillna=True).sma_indicator()
df["sma99"] = SMAIndicator(close=df["Open"], window=99, fillna=True).sma_indicator()

#Adds Bollinger Bands
indicator_bb = BollingerBands(close=df["Open"], window=20, window_dev=2)
df['bb_bbm'] = indicator_bb.bollinger_mavg()
df['bb_bbh'] = indicator_bb.bollinger_hband()
df['bb_bbl'] = indicator_bb.bollinger_lband()

#Adds PSAR Indicator
indicator_psar = PSARIndicator(high=df["Close"], low=df["Low"], close=df["Close"], step=0.02, max_step=2, fillna=True)
df['psar'] = indicator_psar.psar()

# Add Moving Average Convergence Divergence (MACD) indicator
df["MACD"] = macd(close=df["Close"], window_slow=26, window_fast=12, fillna=True)
    
# Add Relative Strength Index (RSI) indicator
df["RSI"] = rsi(close=df["Close"], window=14, fillna=True)

#Indicators using finta
#Adds simple moving average
df["sma40"] = TA.SMA(df, period = 40)

# Add Relative Strength Index (RSI) indicator
df["OBV"] = TA.OBV(df)

#Drops empty spaces
df.fillna(0, inplace=True)

#Drops empty spaces
df.head(50)


/Users/esteban/miniforge3/lib/python3.9/site-packages/ta/trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
/Users/esteban/miniforge3/lib/python3.9/site-packages/ta/trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_down = pd.Series(index=self._psar.index)


,Unix Timestamp,Symbol,Open,High,Low,Close,Volume,sma7,sma25,sma99,bb_bbm,bb_bbh,bb_bbl,psar,MACD,RSI,sma40,OBV
Date,,,,,,,,,,,,,,,,,,
2015-10-08 13:00:00,1444309200,BTCUSD,0.00,245.00,0.00,245.00,0.606654,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,245.000000,0.000000,100.000000,0.00000,0.000000
2015-10-08 14:00:00,1444312800,BTCUSD,245.00,245.00,244.50,245.00,4.453649,122.500000,122.500000,122.500000,0.0000,0.000000,0.000000,245.000000,0.000000,100.000000,0.00000,0.000000
2015-10-08 15:00:00,1444316400,BTCUSD,245.00,245.00,244.92,244.92,3.016926,163.333333,163.333333,163.333333,0.0000,0.000000,0.000000,245.000000,-0.006382,0.000000,0.00000,-3.016926
2015-10-08 16:00:00,1444320000,BTCUSD,244.92,244.92,244.25,244.25,3.895252,183.730000,183.730000,183.730000,0.0000,0.000000,0.000000,245.000000,-0.064756,0.000000,0.00000,-6.912178
2015-10-08 17:00:00,1444323600,BTCUSD,244.25,244.99,244.02,244.99,3.920632,195.834000,195.834000,195.834000,0.0000,0.000000,0.000000,244.250000,-0.050722,51.707665,0.00000,-2.991546
2015-10-08 18:00:00,1444327200,BTCUSD,244.99,244.99,244.00,244.00,3.690472,204.026667,204.026667,204.026667,0.0000,0.000000,0.000000,244.990000,-0.118123,29.632284,0.00000,-6.682018
2015-10-08 19:00:00,1444330800,BTCUSD,244.00,244.00,244.00,244.00,1.531456,209.737143,209.737143,209.737143,0.0000,0.000000,0.000000,244.990000,-0.169584,29.632284,0.00000,0.000000
2015-10-08 20:00:00,1444334400,BTCUSD,244.00,244.00,243.60,244.00,0.177049,244.594286,214.020000,214.020000,0.0000,0.000000,0.000000,244.970200,-0.207969,29.632284,0.00000,0.000000
2015-10-08 21:00:00,1444338000,BTCUSD,244.00,244.00,243.95,243.95,0.041322,244.451429,217.351111,217.351111,0.0000,0.000000,0.000000,244.915392,-0.239662,28.855203,0.00000,-6.723341


In [9]:
#add a column to dataframe

df["Position"]:
    if 
    
    df['Open'] - df['Close']

df

,Unix Timestamp,Symbol,Open,High,Low,Close,Volume,sma7,sma25,sma99,bb_bbm,bb_bbh,bb_bbl,psar,MACD,RSI,sma40,OBV,Position
Date,,,,,,,,,,,,,,,,,,,
2015-10-08 13:00:00,1444309200,BTCUSD,0.00,245.00,0.00,245.00,0.606654,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,245.000000,0.000000,100.000000,0.00000,0.000000,-245.00
2015-10-08 14:00:00,1444312800,BTCUSD,245.00,245.00,244.50,245.00,4.453649,122.500000,122.500000,122.500000,0.0000,0.000000,0.000000,245.000000,0.000000,100.000000,0.00000,0.000000,0.00
2015-10-08 15:00:00,1444316400,BTCUSD,245.00,245.00,244.92,244.92,3.016926,163.333333,163.333333,163.333333,0.0000,0.000000,0.000000,245.000000,-0.006382,0.000000,0.00000,-3.016926,0.08
2015-10-08 16:00:00,1444320000,BTCUSD,244.92,244.92,244.25,244.25,3.895252,183.730000,183.730000,183.730000,0.0000,0.000000,0.000000,245.000000,-0.064756,0.000000,0.00000,-6.912178,0.67
2015-10-08 17:00:00,1444323600,BTCUSD,244.25,244.99,244.02,244.99,3.920632,195.834000,195.834000,195.834000,0.0000,0.000000,0.000000,244.250000,-0.050722,51.707665,0.00000,-2.991546,-0.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-08 20:00:00,1586376000000,BTCUSD,7320.11,7375.72,7313.66,7327.70,153.655811,7295.855714,7273.685600,7097.518485,7293.2500,7413.557949,7172.942051,7387.419017,12.177542,53.946276,7302.99775,109921.940257,-7.59
2020-04-08 21:00:00,1586379600000,BTCUSD,7327.70,7370.00,7311.33,7337.19,35.551553,7310.988571,7274.944000,7102.981313,7301.7945,7405.216715,7198.372285,7383.820636,13.828911,54.889941,7304.31925,109957.491810,-9.49
2020-04-08 22:00:00,1586383200000,BTCUSD,7337.19,7390.25,7322.87,7367.28,88.486108,7320.638571,7283.348400,7108.498485,7310.5595,7392.564183,7228.554817,7380.294224,17.365466,57.839770,7305.70325,110045.977918,-30.09


*2.1*.4. Dump dataset into a pickle file

In [14]:
import pickle
with open("BTCUSD_1day.pickle", 'wb') as f:
      pickle.dump(df,f, pickle.HIGHEST_PROTOCOL)
